In [7]:
import os
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from Dataset import *
from torchvision import transforms


In [2]:

# Function to parse video and extract frames around a specific timestamp
def extract_video_clip(video_path, target_timestamp, video_start_time,time_window=2):
    # Open the video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video: {video_path}")
        return None
    # Get video frame rate and total frame count
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the frame range for the +- time_window
    target_time = target_timestamp
    start_time = target_time - timedelta(seconds=time_window)
    end_time = target_time + timedelta(seconds=time_window)
    if start_time < video_start_time:
        start_time = video_start_time
    if end_time > video_start_time + timedelta(seconds=total_frames / fps):
        end_time = video_start_time + timedelta(seconds=total_frames / fps)
    # print(f"Target Time: {target_time}, Start Time: {start_time}, End Time: {end_time}")
    # Extract frames within the time range
    frames = []
    frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Calculate the timestamp for the current frame
        frame_time = video_start_time + timedelta(seconds=frame_index / fps)
        if start_time <= frame_time <= end_time:
            frames.append(frame)
        frame_index += 1
        if frame_time > end_time:
            break

    cap.release()
    return np.array(frames)

# Load Excel file
def load_excel_data(file_path):
    activation_df = pd.read_excel(file_path, sheet_name="Activation")
    non_activation_df = pd.read_excel(file_path, sheet_name="NonActivation")
    return activation_df, non_activation_df

# Function to process dataset and extract video clips
def process_dataset(video_dir, excel_file, output_dir, clip_save_dir, time_window=3):
    # Load the Excel data
    activation_df, non_activation_df = load_excel_data(excel_file)

    # Create directories for saving data
    os.makedirs(output_dir, exist_ok=True)
    train_dir = os.path.join(output_dir, "train")
    val_dir = os.path.join(output_dir, "val")
    test_dir = os.path.join(output_dir, "test")
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    os.makedirs(clip_save_dir, exist_ok=True)

    # Prepare data for activation and non-activation
    # all_data = []
    for df, label in [(activation_df, 1), (non_activation_df, 0)]:
        for _, row in tqdm(df.iterrows()):
            clip_timestamp = row["timestamp"]
            location = row["location"]
            # convert to datetime object
            clip_timestamp = datetime.strptime(clip_timestamp, "%Y%m%d_%H%M%S")
            # Match the video file with the timestamp
            for video_file in os.listdir(video_dir):
                video_start_timestamp = video_file[22:].split(".")[0]
                # convert to datetime object
                video_start_timestamp = datetime.strptime(video_start_timestamp, "%Y%m%d_%H%M%S")
                video_end_timestamp = video_start_timestamp + timedelta(seconds=60 * 5)
                if video_start_timestamp <= clip_timestamp <= video_end_timestamp:
                    video_path = os.path.join(video_dir, video_file)
                    clip_save_path = os.path.join(clip_save_dir, f"{label}_{row['timestamp']}_{location}.pt")
                    if os.path.exists(clip_save_path):
                        continue
                    # Extract video clip
                    frames = extract_video_clip(video_path, clip_timestamp, video_start_timestamp,time_window)
                    if frames is not None:
                        data = (frames, label, row["timestamp"], location)
                        # save the clip to the clip_save_dir
                        # clip_save_path = os.path.join(clip_save_dir, f"{label}_{row['timestamp']}_{location}.pt")
                        torch.save(torch.tensor(frames), clip_save_path)



In [4]:
# Process the dataset
video_dir = r"D:\LiDAR_Data\2ndPHB\Video\IntesectionVideo"
excel_file = r"D:\LiDAR_Data\2ndPHB\Video\Activation.xlsx"
output_dir = r"D:\LiDAR_Data\2ndPHB\Video\Dataset"
clip_save_dir = r"D:\LiDAR_Data\2ndPHB\Video\Clips"
time_window = 3 # seconds
# Load the Excel data
activation_df, non_activation_df = load_excel_data(excel_file)

# Create directories for saving data
os.makedirs(output_dir, exist_ok=True)
train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "val")
test_dir = os.path.join(output_dir, "test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(clip_save_dir, exist_ok=True)

# Prepare data for activation and non-activation
# all_data = []
for df, label in [(activation_df, 1), (non_activation_df, 0)]:
    for _, row in tqdm(df.iterrows()):
        clip_timestamp = row["timestamp"]
        location = row["location"]
        # convert to datetime object
        clip_timestamp = datetime.strptime(clip_timestamp, "%Y%m%d_%H%M%S")
        # Match the video file with the timestamp
        for video_file in os.listdir(video_dir):
            video_start_timestamp = video_file[22:].split(".")[0]
            # convert to datetime object
            video_start_timestamp = datetime.strptime(video_start_timestamp, "%Y%m%d_%H%M%S")
            video_end_timestamp = video_start_timestamp + timedelta(seconds=60 * 5)
            if video_start_timestamp <= clip_timestamp <= video_end_timestamp:
                video_path = os.path.join(video_dir, video_file)
                clip_save_path = os.path.join(clip_save_dir, f"{label}_{row['timestamp']}_{location}.pt")
                if os.path.exists(clip_save_path):
                    continue
                # Extract video clip
                frames = extract_video_clip(video_path, clip_timestamp, video_start_timestamp,time_window)
                if frames is not None:
                    data = (frames, label, row["timestamp"], location)
                    # save the clip to the clip_save_dir
                    clip_save_path = os.path.join(clip_save_dir, f"{label}_{row['timestamp']}_{location}.mp4")
                    out = cv2.VideoWriter(clip_save_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (frames.shape[2], frames.shape[1]))
                    for frame in frames:
                        out.write(frame)
                    out.release()
# process_dataset(video_dir, excel_file, output_dir,clip_save_dir, time_window=3)

120it [27:08, 13.57s/it]
402it [1:31:02, 13.59s/it]


In [9]:
# Split the dataset into train, validation with 80% and 20% respectively
clip_save_dir = r"D:\LiDAR_Data\2ndPHB\Video\Clips"
clips = os.listdir(clip_save_dir)
train_clips, val_clips = train_test_split(clips, test_size=0.2, random_state=714)
train_folder = r'D:\LiDAR_Data\2ndPHB\Video\Dataset\train'
val_folder = r'D:\LiDAR_Data\2ndPHB\Video\Dataset\val'
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)
# Move the clips to the respective folders
for clip in train_clips:
    src = os.path.join(clip_save_dir, clip)
    dst = os.path.join(train_folder, clip)
    os.rename(src, dst)
for clip in val_clips:
    src = os.path.join(clip_save_dir, clip)
    dst = os.path.join(val_folder, clip)
    os.rename(src, dst)

In [3]:
def create_data_loaders(train_dir, val_dir, batch_size=8, transform=None):
    train_dataset = VideoDataset(train_dir, transform=transform)
    val_dataset = VideoDataset(val_dir, transform=transform)
    # test_dataset = VideoDataset(test_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    # test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

In [6]:
from Model import *
from Dataset import *
import os
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
train_folder = r'D:\LiDAR_Data\2ndPHB\Video\Dataset\train'
val_folder = r'D:\LiDAR_Data\2ndPHB\Video\Dataset\val'
train_loader, val_loader = create_data_loaders(train_folder, val_folder, batch_size=4, transform=custom_transform)

In [3]:
for i, (frames, labels, locations) in enumerate(train_loader):
    print(frames.shape, labels)
    break

torch.Size([2, 61, 3, 224, 224]) tensor([0, 0])


In [ ]:
num_epochs=10
learning_rate=0.001
run_dir = "D:\LiDAR_Data\2ndPHB\Video\run"
model = ResNetLSTM().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

os.makedirs(run_dir, exist_ok=True)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float()

        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    scheduler.step()

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device).float()
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss / len(train_loader)}, Val Loss: {val_loss / len(val_loader)}")

    # Save model checkpoint
    checkpoint_path = os.path.join(run_dir, f"epoch_{epoch + 1}.pth")
    torch.save(model.state_dict(), checkpoint_path)

In [28]:
resnet = models.resnet18(weights = models.ResNet18_Weights.DEFAULT)
feature_extractor = nn.Sequential(*list(resnet.children())[:-1])

In [57]:
batch_size, seq_len, c, h, w = frames.shape
x = frames.view(batch_size * seq_len, c, h, w).permute(0, 3, 1, 2)
features = feature_extractor(x)
features = features.view(batch_size, seq_len, -1)

In [60]:
lstm_hidden_dim = 128
lstm_layers = 2
input_size = 512
lstm = nn.LSTM(input_size=input_size, hidden_size=lstm_hidden_dim, num_layers=lstm_layers, batch_first=True)

In [61]:
lstm_out, _ = lstm(features)

In [63]:
lstm_out = lstm_out[:, -1, :]

torch.Size([4, 61, 128])